In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
data=pd.read_excel('/Users/User/Downloads/spectrum_data_excel.xlsx' , header=None)
data.columns=("f", "x", "y", "p")

# the rolling mean for power/intensity
data['rolp']=data['p'].rolling(15).mean()

#show the rest value for HI frequency
fo = 1420.4

h=np.linspace(data['p'].min(), data['p'].max(), 2048)
x=np.full_like(h,fo)


# find the radial velocity
def u(f):
    c=3*10**5
    return ((fo -f)/fo )*c

data['u']=u(data.f)

def inv(u):
    c=3*10**5
    return fo - (u/c)*fo



#visualise the data
fig, ax1=plt.subplots()
ax1.plot(data.f, data.p, color='midnightblue', label='raw')
ax1.plot(data.f , data.rolp, color='coral', label='rolling mean n=20')
ax1.plot(x,h, color='firebrick', linestyle='--', linewidth =1.5 )
ax1.set_xlabel("Frequency [MHz]")
ax1.set_ylabel("Relative Power")
ax1.yaxis.get_ticklocs(minor=True)
ax1.xaxis.get_ticklocs(minor=True)
ax1.minorticks_on()
ax1.set_box_aspect(1.0)

ax2=ax1.secondary_xaxis('top', functions=(u, inv))
ax2.set_xlabel("Radial Velocity [km/s]")
ax2.xaxis.get_ticklocs(minor=True)


plt.grid(True, linestyle ="solid", c="lightgrey")
plt.title("spectrum for finding HI")
plt.legend(loc="upper center",bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.subplots_adjust(bottom=0.2)
plt.show()